# hCassandra runTests

## Description

---

### Objective

   This test aims to **automate**:
   
   (1) The execution of the Hydra Cassandra Stress Test (hCassandra) for increasing client load.
   
   (2) The generation of performance results presented in the form of tables and graphs for relevant metrics. 
   
   To this end, performance of the Cassandra Cluster is measured as the number of clients writing and reading into the DataBase is increased. The number of clients can be defined by the user. 
   
 
### Customize the Test

   Modify **total_num_clients** to change the sets of clients for which you wish to execute the test.
   
   Current tests have been run for a maximum of **10000** clients and a duration of 5 minutes against a 3-node Cluster (for further details on Software & Hardware specs please refer to the *Software & Hardware Specs* section).
   
### Useful HINTS for running the test

- If test has been previously executed and output is still shown, you can restart (delete former results) by selecting in the top menu Cell -> All Output -> Clear
- To run test, step on top of the code cells and press the 'run cell' button on the top menu. For automatic Run select from the top menu Cell -> Run All
- If you wish to store your results. After RUN is finished, generate your own report by selecting FILE -> Download as -> Markdown (.md) (or any other preferred format).

## Software & Hardware Specs

---

The tests were executed on Google Cloud Servers, with the following specs:

#### Cassandra Cluster

- 3 Node Cluster, each with the following specs:
   - 16 vCPUs
   - RAM: 60 GB
   - Disk: 60 GB
   - OS: Debian 3.16.7-ckt25-2
  
- Cassandra + Cassandra-Tools Version: 3.0.6

#### Hydra Cluster

- **MASTER**: 1 Server
   - 4 vCPUs
   - RAM: 15 GB
   - OS: Ubuntu 14.04

- **SLAVES**: 9 Servers (hosts to the cassandra-stress tool)
   - 16 vCPUs 
   - RAM: 60 GB
   - Disk: 60 GB
   - OS: Debian 3.16.7-ckt25-2
   
### Important 

- For the performance tests maximum file open limit (ulimit) had to be increased for the Master Node.

---

## hCassandra Test 1: Fixed Number of Stress Clients (Debug Mode) 

---

The following test runs a SINGLE execution of the Cassandra Test for a fixed number of clients (total_client_count) and operations (total_ops_count). Runs in debug mode: showing logger info during execution. 

In [ ]:
!python ./src/hCassandra_test.py --cluster_ips='10.10.0.155,10.10.2.218,10.10.3.116' --total_client_count=8000 --total_ops_count=2000

---

## hCassandra Test 2: Increasing the Number of stress clients (multiple runs)

---


**IMPORTANT**:

   If you want to change the number of clients and/or number of operations for your test, please set values to desired in the following section:


In [1]:
# Define num Client(s) / Operation(s) 
total_num_clients = [10, 100, 200, 400, 600, 700, 800, 900]
total_ops_count = [1000000]
# Set IPs of Nodes in Cassandra Cluster
cassandra_cluster_ips = '10.10.2.68,10.10.2.90,10.10.2.193'

**UTIL FUNCTIONS**

In [2]:
import json
import ast

def get_result(test_stdout):
    """This Function gets (filters) the Cassandra Test Results from stdout"""
    index_start = test_stdout.find('Cassandra Stress Results: \n')
    index_end = test_stdout.find('Calling Server shutdown')
    if index_start != -1:
        results = test_stdout[(index_start + len('Cassandra Stress Results: \n')):index_end]
        res_dict = ast.literal_eval(results)
        return res_dict
    else:
        return {}

The following block of code is the actual **EXECUTION OF THE CASSANDRA SCALE TESTS**. This may take a couple of minutes:

In [3]:
import subprocess
import os
import json
import signal

hCassandra_results = dict()

print 'STARTING CASSANDRA STRESS TESTS \n'
# Execute hCassandra_test for given client_count
for idx1, clients in enumerate(total_num_clients):
    for idx2, ops in enumerate(total_ops_count):
        print ('Test (%s/%s) in progress.. Please wait until test is completed..' % ((len(total_ops_count) * idx1) + idx2 + 1,len(total_num_clients) * len(total_ops_count)))
        # Execute hCassandra_test.py (python script for hCassandra Scale Test)
        hcass_cmd = "python ./src/hCassandra_test.py --cluster_ips=%s --total_client_count=%s --total_ops_count=%s" % (cassandra_cluster_ips, clients, ops)
        stress_test = subprocess.Popen(hcass_cmd, stdout=subprocess.PIPE,
                                          stderr=subprocess.PIPE, shell=True, preexec_fn=os.setsid)
        stdout, stderr = stress_test.communicate()
        results_dict = get_result(stdout)
        if len(results_dict) <= 1:
            print ('There was an ERROR while attempting to parse stdout...')
            print 'STDOUT: %s' % stdout
            print 'STDERR: %s' % stderr
        if not str(clients) in hCassandra_results:
            hCassandra_results[str(clients)] = dict()
        hCassandra_results[str(clients)][str(ops)] = results_dict
        print 'Test SUCCESFULLY completed... \n'

print 'END OF TESTS:'
print 'ALL TESTS HAVE BEEN COMPLETED. PLEASE PROCEED TO GENERATE GRAPHS & TABLES WITH PERFORMANCE RESULTS.'

STARTING CASSANDRA STRESS TESTS 

Test (1/8) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (2/8) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (3/8) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (4/8) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (5/8) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (6/8) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (7/8) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (8/8) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

END OF TESTS:
ALL TESTS HAVE BEEN COMPLETED. PLEASE PROCEED TO GENERATE GRAPHS & TABLES WITH PERFORMANCE RESULTS.


  **NOTE:**
  
  ---
   
   Wait until RESULTS (**hCassandra_results**) are generated for all cases, and then execute the following blocks to generate:
   (1) Tables with results (markdown compatible) and 
   (2) Graphs.
   
   The **END OF TEST** is indicated by a message. Please wait...
   
   ---

### RESULT PROCESSING & TABLE/ GRAPH GENERATION

---

In this section, we process the results for generating tables with performance values and graphs that reflect number of operations per second and median latency for increased number of clients. 

**NOTE**
If you are interested in representing any other performance metric, follow the pattern followed for any of the two graps already provided. 

In [4]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [5]:
results_per_ops = dict()

# Table Format: Metrics
header = [
            '# Clients',
            'total_ops',
            'op/s',
            'pk/s',
            'med',
            '.95',
            '.99',
            'max',
            'max_ms',
            'sdv_ms',
            'op_time'
        ]

data_matrix_write = ListTable()
data_matrix_read = ListTable()

data_matrix_write.append(header)
data_matrix_read.append(header)

results_per_ops[str(total_ops_count[0])] = dict()
for clients in total_num_clients:
    if str(clients) in hCassandra_results:
        res_dict = ast.literal_eval(hCassandra_results[str(clients)][str(ops)])
        results_per_ops[str(ops)][str(clients)] = res_dict
        data_matrix_write.append([clients, res_dict['write']['total ops'], res_dict['write']['op/s'], res_dict['write']['pk/s'], res_dict['write']['med'], res_dict['write']['.95'], res_dict['write']['.99'], res_dict['write']['max'], res_dict['write']['max_ms'], res_dict['write']['sdv_ms'], res_dict['write']['op_time']])
        data_matrix_read.append([clients, res_dict['read']['total ops'], res_dict['read']['op/s'], res_dict['read']['pk/s'], res_dict['read']['med'], res_dict['read']['.95'], res_dict['read']['.99'], res_dict['read']['max'], res_dict['read']['max_ms'], res_dict['read']['sdv_ms'], res_dict['read']['op_time']])

### Result Generation: Table Format

Next, results are displayed in a Table, following the markdown format..  



The next table represents the results for the **WRITE** Operations:

---

*Table 1. "Cassandra Performance over WRITE Operation."*

In [6]:
data_matrix_write

# Clients,total_ops,op/s,pk/s,med,.95,.99,max,max_ms,sdv_ms,op_time
10,[6303597],[21012],[21012],[0.4],[0.6],[0.9],[103.3],[],[0.0],[' 00:05:00']
100,"[4713821, 4224073, 4788731, 4184406, 4047217]","[15713, 14080, 15962, 13948, 13491]","[15713, 14080, 15962, 13948, 13491]","[0.7, 1.0, 0.7, 1.0, 1.0]","[2.9, 2.8, 2.7, 2.8, 3.0]","[6.4, 5.5, 5.7, 5.5, 6.0]","[184.7, 183.6, 188.5, 185.2, 185.2]",[],"[0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:04:59', ' 00:04:59']"
200,"[2650759, 2640611, 2593114, 2731517, 2955856, 2721583, 2859340, 2653934, 2873350, 2667175]","[8836, 8802, 8643, 9105, 9853, 9072, 9531, 8846, 9577, 8891]","[8836, 8802, 8643, 9105, 9853, 9072, 9531, 8846, 9577, 8891]","[1.5, 1.4, 1.4, 1.4, 1.3, 1.4, 1.3, 1.4, 1.3, 1.4]","[5.2, 5.3, 5.3, 5.1, 4.6, 5.2, 5.0, 5.2, 4.8, 5.1]","[9.9, 10.0, 10.0, 9.8, 8.9, 10.0, 10.0, 10.0, 9.3, 9.5]","[246.6, 207.8, 249.0, 248.0, 207.6, 247.4, 202.8, 247.8, 249.0, 249.2]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:05:00', ' 00:04:59', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:04:59']"
400,"[1548970, 1475201, 1493593, 1535292, 1533274, 1539914, 1493496, 1503966, 1533236, 1512055, 1536969, 1543964, 1525365, 1541306, 1530848, 1535301, 1485453, 1485216, 1500342, 1523987]","[5164, 4917, 4979, 5117, 5110, 5133, 4978, 5013, 5111, 5040, 5123, 5146, 5084, 5137, 5103, 5117, 4951, 4951, 5001, 5080]","[5164, 4917, 4979, 5117, 5110, 5133, 4978, 5013, 5111, 5040, 5123, 5146, 5084, 5137, 5103, 5117, 4951, 4951, 5001, 5080]","[2.3, 2.5, 2.4, 2.3, 2.4, 2.4, 2.4, 2.4, 2.3, 2.4, 2.3, 2.3, 2.4, 2.3, 2.3, 2.4, 2.5, 2.5, 2.4, 2.3]","[9.8, 10.4, 10.3, 9.8, 10.4, 10.1, 10.3, 10.3, 9.8, 10.1, 10.0, 10.0, 10.5, 9.8, 10.3, 9.8, 10.2, 10.2, 10.3, 10.1]","[16.8, 17.4, 17.8, 16.6, 18.4, 17.6, 17.2, 17.7, 16.4, 17.3, 17.0, 17.3, 18.0, 16.6, 17.7, 16.4, 17.4, 17.3, 17.2, 17.2]","[201.4, 202.7, 198.7, 202.9, 200.4, 201.5, 203.2, 204.3, 195.6, 201.1, 200.9, 197.8, 215.0, 201.8, 200.4, 208.1, 199.0, 203.5, 201.2, 202.8]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:04:59', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:04:59', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:04:59', ' 00:05:00']"
600,"[1020913, 983628, 986631, 997303, 975142, 1012283, 988408, 999895, 999817, 983857, 967786, 987243, 1002855, 1014505, 1013734, 1015615, 980063, 1025754, 998142, 996638, 1003766, 986446, 997961, 987422, 1002423, 984585, 998228, 984698, 1008231, 1004009]","[3403, 3279, 3289, 3324, 3250, 3374, 3295, 3333, 3333, 3280, 3226, 3291, 3343, 3382, 3379, 3386, 3267, 3419, 3327, 3322, 3346, 3288, 3326, 3291, 3341, 3282, 3327, 3282, 3361, 3346]","[3403, 3279, 3289, 3324, 3250, 3374, 3295, 3333, 3333, 3280, 3226, 3291, 3343, 3382, 3379, 3386, 3267, 3419, 3327, 3322, 3346, 3288, 3326, 3291, 3341, 3282, 3327, 3282, 3361, 3346]","[3.2, 3.5, 3.5, 3.3, 3.6, 3.2, 3.5, 3.3, 3.4, 3.5, 3.6, 3.5, 3.3, 3.2, 3.3, 3.2, 3.5, 3.2, 3.4, 3.3, 3.3, 3.5, 3.3, 3.5, 3.4, 3.5, 3.3, 3.4, 3.2, 3.3]","[16.1, 16.2, 16.1, 16.5, 16.1, 16.4, 16.2, 16.2, 16.6, 16.6, 16.9, 16.1, 16.7, 16.7, 16.8, 16.4, 16.8, 16.5, 16.7, 16.8, 16.4, 16.3, 16.4, 16.4, 16.5, 16.3, 16.4, 16.7, 16.2, 16.6]","[28.0, 28.0, 28.1, 29.2, 27.7, 29.7, 26.7, 28.5, 28.6, 28.6, 28.9, 27.8, 29.2, 28.6, 29.5, 28.8, 28.3, 28.8, 29.2, 29.3, 28.1, 27.2, 28.8, 27.5, 28.2, 27.7, 29.2, 29.2, 27.8, 28.0]","[293.1, 257.6, 286.3, 253.1, 256.3, 274.3, 256.2, 238.3, 250.7, 285.8, 265.6, 264.3, 266.6, 256.8, 262.3, 284.5, 257.8, 276.7, 361.6, 261.6, 259.7, 249.5, 252.9, 255.4, 255.7, 254.9, 274.0, 234.9, 271.4, 257.1]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', '

The next table represents the results for the **READ** Operations:

---

*Table 1. "Cassandra Performance over READ Operation."*

In [7]:
data_matrix_read

# Clients,total_ops,op/s,pk/s,med,.95,.99,max,max_ms,sdv_ms,op_time
10,[2534164],[21118],[21118],[0.4],[0.7],[1.0],[47.4],[],[0.0],[' 00:02:00']
100,"[1812527, 1547037, 1610967, 1549853, 1400377]","[15104, 12891, 13425, 12915, 11670]","[15104, 12891, 13425, 12915, 11670]","[0.8, 1.1, 0.9, 1.1, 1.1]","[2.9, 3.3, 3.7, 3.3, 4.2]","[6.2, 6.6, 8.0, 6.4, 8.3]","[71.2, 68.9, 76.5, 61.0, 74.7]",[],"[0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:02:00', ' 00:02:00', ' 00:01:59', ' 00:02:00', ' 00:02:00']"
200,"[908674, 1013038, 948513, 905372, 907947, 884722, 970653, 876564, 875239, 889936]","[7573, 8441, 7903, 7546, 7565, 7372, 8087, 7305, 7293, 7416]","[7573, 8441, 7903, 7546, 7565, 7372, 8087, 7305, 7293, 7416]","[1.7, 1.5, 1.6, 1.6, 1.6, 1.6, 1.5, 1.7, 1.7, 1.7]","[6.4, 5.7, 6.0, 6.5, 6.9, 7.1, 6.0, 7.3, 7.0, 6.7]","[12.8, 11.8, 11.4, 12.9, 13.8, 14.2, 12.5, 14.6, 14.0, 13.1]","[204.2, 209.0, 204.9, 205.7, 219.6, 204.9, 205.9, 204.1, 207.9, 209.8]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:01:59', ' 00:02:00', ' 00:02:00', ' 00:01:59', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00']"
400,"[545592, 521882, 533478, 543833, 561765, 537182, 526971, 531536, 542187, 538936, 546755, 543805, 549632, 552979, 546018, 540239, 526256, 536026, 545983, 539742]","[4546, 4348, 4445, 4532, 4681, 4476, 4392, 4429, 4518, 4490, 4557, 4531, 4580, 4608, 4550, 4502, 4386, 4467, 4550, 4497]","[4546, 4348, 4445, 4532, 4681, 4476, 4392, 4429, 4518, 4490, 4557, 4531, 4580, 4608, 4550, 4502, 4386, 4467, 4550, 4497]","[2.6, 2.9, 2.8, 2.7, 2.5, 2.7, 2.8, 2.8, 2.7, 2.7, 2.6, 2.7, 2.6, 2.6, 2.7, 2.6, 2.8, 2.7, 2.6, 2.7]","[11.6, 12.2, 12.5, 11.9, 11.8, 12.1, 12.5, 12.2, 12.5, 11.9, 12.4, 12.0, 12.0, 11.7, 12.0, 12.0, 12.0, 12.1, 12.1, 12.5]","[20.1, 21.1, 21.2, 21.1, 20.7, 21.2, 21.7, 20.8, 22.1, 20.3, 21.8, 20.7, 20.0, 19.8, 20.6, 20.7, 20.3, 20.2, 20.4, 21.9]","[239.3, 140.8, 191.3, 238.2, 231.1, 223.8, 208.1, 166.3, 246.3, 267.0, 246.0, 171.8, 225.3, 133.9, 242.6, 223.0, 227.0, 247.5, 181.6, 180.3]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:01:59', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:01:59', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:01:59', ' 00:02:00', ' 00:02:00', ' 00:02:00']"
600,"[374143, 361582, 366095, 353354, 361022, 374844, 361883, 355384, 368422, 369757, 362892, 362951, 358920, 375157, 370368, 367930, 365236, 361771, 356046, 372573, 377193, 360256, 357594, 356567, 359306, 365396, 355781, 359242, 375417, 357253]","[3118, 3013, 3050, 2945, 3008, 3123, 3015, 2961, 3070, 3081, 3022, 3024, 2990, 3126, 3086, 3065, 3044, 3014, 2967, 3104, 3143, 3001, 2980, 2971, 2993, 3045, 2965, 2994, 3127, 2977]","[3118, 3013, 3050, 2945, 3008, 3123, 3015, 2961, 3070, 3081, 3022, 3024, 2990, 3126, 3086, 3065, 3044, 3014, 2967, 3104, 3143, 3001, 2980, 2971, 2993, 3045, 2965, 2994, 3127, 2977]","[3.5, 3.9, 3.8, 3.9, 3.9, 3.6, 3.8, 3.9, 3.6, 3.8, 3.9, 3.9, 3.9, 3.6, 3.6, 3.6, 3.9, 3.7, 3.9, 3.6, 3.5, 3.8, 3.9, 4.0, 3.8, 3.8, 3.8, 3.8, 3.6, 3.8]","[18.7, 18.6, 18.4, 19.0, 19.1, 19.3, 19.2, 19.2, 19.1, 18.1, 18.9, 18.2, 19.4, 18.7, 18.8, 18.8, 18.1, 19.9, 19.3, 18.5, 18.0, 18.9, 19.3, 18.5, 19.0, 18.7, 19.3, 19.6, 18.7, 19.2]","[31.3, 31.4, 30.9, 32.4, 32.2, 33.3, 33.4, 33.0, 32.7, 30.3, 31.8, 31.4, 34.0, 33.1, 32.3, 31.3, 30.9, 32.5, 33.2, 31.0, 30.6, 31.1, 33.1, 31.4, 32.4, 31.9, 33.0, 33.4, 32.2, 33.0]","[487.7, 534.1, 485.8, 541.5, 490.4, 484.9, 483.5, 493.8, 485.3, 489.2, 539.5, 487.4, 491.9, 484.1, 490.0, 480.9, 523.6, 483.2, 484.8, 486.1, 485.4, 485.6, 492.0, 542.2, 482.2, 485.2, 482.9, 480.2, 538.5, 509.5]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:01:59', ' 00:02:00', ' 00:02:00', ' 00:02:00'

### Result Generation: Graphs

Next, results are displayed in Graphs. 

--- 

**IMPORTANT**

Please, MODIFY the graphs name here if desired. Otherwise, graphs are indexed by datetime. 



---

In [8]:
from datetime import datetime

ops_second_graph_filename = "hCassandra_ops_" + str(datetime.now().strftime("%m%d%Y_%H%M%S"))
median_latency_graph_filename = "hCassandra_med_" + str(datetime.now().strftime("%m%d%Y_%H%M%S"))

In [9]:
%%capture

import plotly.plotly as py
from plotly.graph_objs import *
import operator
import numpy
import collections

data_matrix = [['# ops', '# Clients', 'total_ops', 'op/s', 'pk/s', 'med', '.95', '.99', 'max', 'max_ms', 'sdv_ms', 'op_time']]


traces_plot1 = []
traces_plot2 = []

# For each trace = client count
for ops_count, tests_per_trace in results_per_ops.iteritems():
    
    total_ops = []
    op_s = []
    op_s_r = []
    pk_s = []
    med = []
    med_r = []
    p95 = []
    p99 = []
    max_lat = []
    max_ms = []
    sdv_ms = []
    op_time = []
    
    # Sort by num_clients

    od = collections.OrderedDict(sorted(tests_per_trace.items()))
    
    for num_clients, res_dict in od.iteritems():
        op_s.append(sum((ops for ops in res_dict['write']['op/s'])))
        med.append(numpy.median(res_dict['write']['med']))
        op_s_r.append(sum((ops for ops in res_dict['read']['op/s'])))
        med_r.append(numpy.median(res_dict['read']['med']))
        # Un-Comment for any other feature you want to depict...
        
        #total_ops.append(sum((ops for ops in res_dict['write']['total ops'])))
        #pk_s.append(res_dict['write']['pk/s'])
        #p95.append(res_dict['write']['.95'])
        #p99.append(res_dict['write']['.99'])
        #max_lat.append(res_dict['write']['max'])
        #max_ms.append(res_dict['write']['max_ms'])
        #sdv_ms.append(res_dict['write']['sdv_ms'])
        #op_time.append(res_dict['write']['op_time'])
        
        
    trace_plot1 = Scatter(
          x=total_num_clients,
          y=op_s, 
          mode = 'lines+markers',
          name = 'WRITE',
          line=dict(
            shape='spline'
            )
        )
    
    trace_plot2 = Scatter(
          x=total_num_clients,
          y=op_s_r, 
          mode = 'lines+markers',
          name = 'READ',
          line=dict(
            shape='spline'
            )
        )
        
    trace_plot3 = Scatter(
          x=total_num_clients,
          y=med, 
          mode = 'lines+markers',
          name = 'WRITE', 
          line=dict(
            shape='spline'
            )
        )

    trace_plot4 = Scatter(
          x=total_num_clients,
          y=med_r, 
          mode = 'lines+markers',
          name = 'READ', 
          line=dict(
            shape='spline'
            )
        )
    
    traces_plot1.append(trace_plot1)
    traces_plot1.append(trace_plot2)
    traces_plot2.append(trace_plot3)
    traces_plot2.append(trace_plot4)


### Result Generation: operations per second vs. client count

The following graph illustrates how, the number of operations per second changes while the number of clients increases 

In [10]:
%%capture plot_operations --no-stdout

data = Data(traces_plot1)
# Edit the layout
layout = dict(title = 'op/s vs. # Clients',
              xaxis = dict(title = '# clients'),
              yaxis = dict(title = 'op/s'),
              )

# Plot and embed in notebook
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = ops_second_graph_filename)

### Result Generation: median latency vs. client count

The following graph illustrates median latency in miliseconds for each operation during that run as the number of clients increases. 

In [11]:
%%capture plot_med --no-stdout

data = Data(traces_plot2)
# Edit the layout
layout = dict(title = 'Median Latency vs. Client Count',
              xaxis = dict(title = '# Clients'),
              yaxis = dict(title = 'Median Latency [ms]'),
              )

# Plot and embed in notebook
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = median_latency_graph_filename)

Overall, these benchmarks represent the **maximum throughput** of a 3 node cluster for the *default* model generated by the cassandra-stress tool. For accurate performance assessment of an application a range of parameters (including data model, queries, etc.) need to be adjusted. 